In [1]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import cv2
from tqdm import tqdm

from load_data import BiopsyDataset, get_balanced_dataloader, Nullmutation
from eval import get_accuracy_per_class

DATA_DIR = "../../data/biopsies_s1.0_anon_data/"
PATHXL_DIR = os.path.join(DATA_DIR, "..", "p53_consensus_study")
BOLERO_DIR = os.path.join(DATA_DIR, "..", "BOLERO")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print("Device: {}".format(device))

# Load csv
csv_file = os.path.join(DATA_DIR, "biopsy_labels_anon_s1.0.csv")
main_df = pd.read_csv(csv_file)

pathxl_csv_file = os.path.join(PATHXL_DIR, "labels.csv")
pathxl_df = pd.read_csv(pathxl_csv_file)

bolero_files = os.listdir(os.path.join(BOLERO_DIR, "biopsies"))
bolero_indices = [f.split(".")[0].split('_') for f in bolero_files]
bolero_df = pd.DataFrame(bolero_indices, columns=["case", "biopsy"])

C:\Users\321lu\AppData\Local\Temp\ipykernel_19316\1849884700.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Device: cuda


In [2]:
# Load ResNet18
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)
# ResNet18 is pretrained on images with size 224x224
num_ftrs = resnet18.fc.in_features
print("Number of features: {}".format(num_ftrs))
resnet18.fc = nn.Identity()

resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT).to(device)
# ResNet50 is pretrained on images with size 224x224
num_ftrs = resnet50.fc.in_features
print("Number of features: {}".format(num_ftrs))
resnet50.fc = nn.Identity()

Number of features: 512
Number of features: 2048


In [3]:
from resnet import ResNetModel
checkpoint_path = os.path.join(DATA_DIR, "..", "..", "models", "fb_spacing4", "acc0.74_epoch42_s4_e50_end-to-end_ks.ckpt")
# Load lightning model from checkpoint
resnet18_tuned = ResNetModel.load_from_checkpoint(checkpoint_path).to(device)
resnet18_tuned.eval()
resnet18_tuned.model.fc = nn.Identity()

Device: cuda
MODEL: ResNetModel
MODEL ARGS: num_classes=4, latents_path=None, rotation_invariant=True, lr=0.001, weight_decay=0.0005, lr_step_size=30, lr_gamma=0.1


In [16]:
from _resnet_patch import ResNetModel
checkpoint_path = os.path.join(DATA_DIR, "..", "..", "models", "_other", "acc0.73_epoch49_patch_end-to-end.ckpt")
# Load lightning model from checkpoint
retccl_tuned = ResNetModel.load_from_checkpoint(checkpoint_path).to(device)
retccl_tuned.eval()
retccl_tuned.model.fc = nn.Identity()

MODEL: ResNetModel
MODEL ARGS: num_classes=4, lr=0.001, weight_decay=0.0005, lr_step_size=30, lr_gamma=0.1


In [10]:
from RetCCL.custom_objects import retccl_resnet50, HistoRetCCLResnet50_Weights

retccl = retccl_resnet50(weights=HistoRetCCLResnet50_Weights.RetCCLWeights).to(device)
# retccl is pretrained on images with size 256x256 at 1 micron per pixel
num_ftrs = retccl.fc.in_features
print("Number of features: {}".format(num_ftrs))
retccl.fc = nn.Identity()

Number of features: 2048


In [17]:
"""
df:
	id	label
0	0	2
1	1	2
2	2	2
3	3	2
4	4	2
...	...	...
1527	1527	2
1528	1528	2
1529	1529	0
1530	1530	0
1531	1531	0
"""


# ImageNet normalization
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]
TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                      std=[0.229, 0.224, 0.225]) # What this does is it subtracts the mean from each channel and divides by the std
])

TRANSFORM_NORMAL = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], 
                                      std=[0.5, 0.5, 0.5]) # What this does is it subtracts the mean from each channel and divides by the std
])

model_configs = {
    "resnet18": {
        "model": resnet18,
        "size": 224,
        "num_ftrs": 512,
        "transform": TRANSFORM,
        "spacing": 1.0, # Not actually fine-tuned on any spacing
    },
    "resnet50": {
        "model": resnet50,
        "size": 224,
        "num_ftrs": 2048,
        "transform": TRANSFORM,
        "spacing": 1.0, # Not actually fine-tuned on any spacing
    },
    "retccl": {
        "model": retccl,
        "size": 256,
        # "size": None,
        "num_ftrs": 2048,
        "transform": TRANSFORM,
        "spacing": 1.0,
    },
    "retccl_tuned": {
        "model": retccl_tuned,
        "size": 256,
        # "size": None,
        "num_ftrs": 2048,
        "transform": TRANSFORM,
        "spacing": 1.0,
    },
    "resnet18_tuned": {
        "model": resnet18_tuned, # Fine-tuned on FB dataset at 4mpp
        "size": 64,
        "num_ftrs": 512,
        "transform": TRANSFORM_NORMAL,
        "spacing": 4.0,
    }
}

# Full biopsies

In [ ]:
spacing = 4

for model_name, model_config in model_configs.items():
    if model_name != "retccl":
        continue

    model = model_config["model"]
    model.eval()
    size = model_config["size"]
    if size == None:
        transform = TRANSFORM
        size = "Full"
    else:
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((size, size)),
            TRANSFORM
        ])
    num_ftrs = model_config["num_ftrs"]

    # Open image from dataset
    latents = torch.zeros((len(df), 4, num_ftrs)) # 4 is the number of rotations
    latent_file = os.path.join(DATA_DIR, f"latents_s{spacing}_{model_name}.pt")
    if os.path.exists(latent_file):
        latents = torch.load(latent_file).detach()

    for idx, label in tqdm(df.values):
        if torch.any(latents[idx] != 0):
            continue

        img_path = os.path.join(DATA_DIR, "biopsies", f"{idx}.png")
        img = plt.imread(img_path) # HxWxC
        img = torch.tensor(img).permute(2,0,1)

        new_size = (img.shape[1]//spacing, img.shape[2]//spacing)
        # Apply resize transform
        img = torchvision.transforms.functional.resize(img, new_size)

        # Apply other transform
        img = transform(img).unsqueeze(0).float().to(device)

        # for i in range(4):
        #     img_rot = torch.rot90(img, i, [2,3])
        #     img_latent = model(img_rot)
        #     latents[idx, i] = img_latent.detach()

        with torch.no_grad():
            img_latent = model(img)
            latents[idx] = img_latent

        torch.save(latents, latent_file)

# Patches

In [14]:
from load_data import BagDataset, P53_CLASS_NAMES

# model_name = "resnet18_tuned"
model_name = "retccl_tuned"
model_config = model_configs[model_name]
model = model_config["model"]
model.eval()
model = model.to(device)

size = model_config["size"]
# size = 128
resize_factor = 1 / model_config["spacing"]
transform = model_config["transform"]
quantile_range_threshold = 0.1
root_dir = DATA_DIR

df = main_df


oe_patch_indices_file = os.path.join(root_dir, f"oe_patch_indices_gs256.pt")
nm_patch_indices_file = os.path.join(root_dir, f"nm_patch_indices_gs256.pt")
oe_patch_indices_by_biopsy = torch.load(oe_patch_indices_file)
nm_patch_indices_by_biopsy = torch.load(nm_patch_indices_file)
wt_patch_inclusion_p = 1/16


num_ftrs = model_config["num_ftrs"]

indices_filename = os.path.join(root_dir, "non_empty_patch_indices_gs256.pt")
non_empty_patch_indices_by_biopsy = torch.load(indices_filename)

# Load latents
latent_file = os.path.join(root_dir, f"bag_latents_gs{size}_{model_name}.pt")
# latent_file_backup = os.path.join(DATA_DIR, f"bag_latents_gs{size}_{model_name}_backup.pt")
if os.path.exists(latent_file):
    bag_latents = torch.load(latent_file)
else:
    bag_latents = {}
for idx, label in tqdm(df.values):
    if idx in bag_latents:
        continue
    # if idx in non_empty_patch_indices_by_biopsy:
    #     continue

    img_file = os.path.join(root_dir, "biopsies", f"{idx}.png")
    img = plt.imread(img_file)
    img = torch.tensor(img).permute(2, 0, 1).float() # (3, h, w)

    # Resize the img so that the height and width are multiples of grid_spacing, 
    # rounded to the nearest multiple.
    # This prevents the last row and column of patches from being cut off.
    # We also resize the image by the resize_factor instead of each patch separately
    new_size = (max(round(img.shape[1]*resize_factor/size), 1)*size, 
                max(round(img.shape[2]*resize_factor/size), 1)*size)
    img = torch.nn.functional.interpolate(img.unsqueeze(0), size=new_size, 
        mode='bilinear', align_corners=False)[0]

    patches = torch.nn.functional.unfold(img.unsqueeze(0), 
        kernel_size=(size, size), 
        stride=(size, size)) # (1, 3*size*size, n_patches)
    patches = patches.permute(0, 2, 1).reshape(-1, 3, size, size) # (n_patches, 3, size, size)

    non_empty_patch_indices = non_empty_patch_indices_by_biopsy[idx].tolist()

    # # Save patch with wt inclusion probability
    # for i, patch_idx in enumerate(non_empty_patch_indices):
    #     if np.random.rand() > wt_patch_inclusion_p:
    #         continue

    #     patch = patches[patch_idx].cpu().numpy().transpose(1, 2, 0)
    #     patch = (patch * 255).astype(np.uint8)
    #     patch_file = os.path.join(root_dir, "patches", f"{idx}_{patch_idx}.png")
    #     # Convert to BGR
    #     patch = cv2.cvtColor(patch, cv2.COLOR_RGB2BGR)
    #     cv2.imwrite(patch_file, patch)

    # oe_patch_indices = oe_patch_indices_by_biopsy[idx] if idx in oe_patch_indices_by_biopsy else []
    # nm_patch_indices = nm_patch_indices_by_biopsy[idx] if idx in nm_patch_indices_by_biopsy else []
    # oe_patch_indices = list(set(non_empty_patch_indices).intersection(oe_patch_indices))
    # nm_patch_indices = list(set(non_empty_patch_indices).intersection(nm_patch_indices))
    # # Take union of OE and NM patches
    # oe_nm_patch_indices = list(set(oe_patch_indices).union(nm_patch_indices))
    # # Save OE and NM patches
    # for i, patch_idx in enumerate(oe_nm_patch_indices):
    #     patch = patches[patch_idx].cpu().numpy().transpose(1, 2, 0)
    #     patch = (patch * 255).astype(np.uint8)
    #     patch_file = os.path.join(root_dir, "patches", f"{idx}_{patch_idx}.png")
    #     # Convert to BGR
    #     patch = cv2.cvtColor(patch, cv2.COLOR_RGB2BGR)
    #     cv2.imwrite(patch_file, patch)
    
    # Remove patches with low difference between the 75th and 1st percentile
    qranges = torch.quantile(patches.view(patches.shape[0], -1), 0.75, dim=-1) - \
              torch.quantile(patches.view(patches.shape[0], -1), 0.01, dim=-1)
    non_empty_idx = torch.where(qranges >= quantile_range_threshold)[0]
    patches = transform(patches[non_empty_idx])
    patches = patches.to(device)

    # non_empty_patch_indices_by_biopsy[idx] = non_empty_idx

    with torch.no_grad():
        patch_latents = model(patches) # (n_patches, num_ftrs)
    bag_latents[idx] = patch_latents.cpu()[:, None, :] # (n_patches, 1, num_ftrs)

    if idx % 10 == 0:
        torch.save(bag_latents, latent_file)
        # torch.save(bag_latents, latent_file_backup)
    #     torch.save(non_empty_patch_indices_by_biopsy, indices_filename)

    # Clear memory
    del img, patches, patch_latents
    torch.cuda.empty_cache()

torch.save(bag_latents, latent_file)
# torch.save(bag_latents, latent_file_backup)
# torch.save(non_empty_patch_indices_by_biopsy, indices_filename)

100%|██████████| 1532/1532 [1:33:30<00:00,  3.66s/it]


In [ ]:
bag_latents = torch.load(os.path.join(DATA_DIR, "bag_latents_gs256_retccl.pt"))

print(type(bag_latents))
print(len(bag_latents))
print(bag_latents.keys())
print(bag_latents[0].shape)

In [18]:
# PathXL dataset

from load_data import BagDataset, P53_CLASS_NAMES

size = 256
quantile_range_threshold = 0.1
# root_dir = PATHXL_DIR
# df = pathxl_df
root_dir = BOLERO_DIR
df = bolero_df

model_name = "retccl_tuned"
model_config = model_configs[model_name]
resize_factor = 1 / model_config["spacing"]
transform = model_config["transform"]
model = model_config["model"]
model.eval()
model = model.to(device)
num_ftrs = model_config["num_ftrs"]

# indices_filename = os.path.join(PATHXL_DIR, "non_empty_patch_indices_gs256.pt")
# if os.path.exists(indices_filename):
#     non_empty_patch_indices_by_biopsy = torch.load(indices_filename)
# else:
#     non_empty_patch_indices_by_biopsy = {}

# Load latents
latent_file = os.path.join(root_dir, f"bag_latents_gs{size}_{model_name}.pt")
if os.path.exists(latent_file):
    bag_latents = torch.load(latent_file)
else:
    bag_latents = {}
for i, row in tqdm(enumerate(df.values), total=len(df)):
    idx = f"{row[0]}_{row[1]}"
    # if idx in bag_latents and idx in non_empty_patch_indices_by_biopsy:
    #     continue
    if idx in bag_latents:
        continue

    img_file = os.path.join(root_dir, "biopsies", f"{idx}.png")
    if not os.path.exists(img_file):
        print(f"File {img_file} does not exist")
        continue
    img = plt.imread(img_file)
    img = torch.tensor(img).permute(2, 0, 1).float() # (3, h, w)

    # Resize the img so that the height and width are multiples of grid_spacing, 
    # rounded to the nearest multiple.
    # This prevents the last row and column of patches from being cut off.
    # We also resize the image by the resize_factor instead of each patch separately
    new_size = (max(round(img.shape[1]*resize_factor/size), 1)*size, 
                max(round(img.shape[2]*resize_factor/size), 1)*size)
    img = torch.nn.functional.interpolate(img.unsqueeze(0), size=new_size, 
        mode='bilinear', align_corners=False)[0]

    patches = torch.nn.functional.unfold(img.unsqueeze(0), 
        kernel_size=(size, size), 
        stride=(size, size)) # (1, 3*size*size, n_patches)
    patches = patches.permute(0, 2, 1).reshape(-1, 3, size, size) # (n_patches, 3, size, size)
    
    # Remove patches with low difference between the 75th and 1st percentile
    qranges = torch.quantile(patches.view(patches.shape[0], -1), 0.75, dim=-1) - \
              torch.quantile(patches.view(patches.shape[0], -1), 0.01, dim=-1)
    non_empty_idx = torch.where(qranges >= quantile_range_threshold)[0]
    patches = transform(patches[non_empty_idx])
    patches = patches.to(device)

    # non_empty_patch_indices_by_biopsy[idx] = non_empty_idx

    with torch.no_grad():
        patch_latents = model(patches) # (n_patches, num_ftrs)
    bag_latents[idx] = patch_latents.cpu()[:, None, :] # (n_patches, 1, num_ftrs)

    if i % 10 == 0:
        torch.save(bag_latents, latent_file)
        # torch.save(non_empty_patch_indices_by_biopsy, indices_filename)

    # Clear memory
    del img, patches, patch_latents
    torch.cuda.empty_cache()

torch.save(bag_latents, latent_file)
# torch.save(non_empty_patch_indices_by_biopsy, indices_filename)

  0%|          | 0/226 [00:00<?, ?it/s]

100%|██████████| 226/226 [12:37<00:00,  3.35s/it]


# Test how padding affects the output

In [ ]:
model = model_configs["resnet18"]["model"]

img_path = os.path.join(DATA_DIR, "biopsies", f"{df.iloc[0,0]}.png")
img = plt.imread(img_path) # HxWxC
plt.imshow(img)
plt.show()

img = torch.tensor(img).permute(2,0,1).unsqueeze(0).float().to(device)
img_latent = model(img)
img_latent_padded = {}
pad = 0
if img.shape[2] != img.shape[3]:
    pad = abs(img.shape[2] - img.shape[3]) // 2
for pad_value in [0, 0.5, 1]:
    if img.shape[2] > img.shape[3]:
        img_pad = torch.nn.functional.pad(img, (pad, pad,0,0), value=pad_value)
    else:
        img_pad = torch.nn.functional.pad(img, (0,0,pad,pad), value=pad_value)

    # Show again
    show_img = img_pad.squeeze().permute(1,2,0).cpu().numpy()
    plt.imshow(show_img)
    plt.show()

    img_latent_padded[pad_value] = model(img_pad)

In [ ]:
for img_latent_pad in img_latent_padded.values():
    # print(img_latent_pad - img_latent)

    # Print MSE between the two
    print(torch.nn.functional.mse_loss(img_latent, img_latent_pad))

    # Did the padding change the latent representation?
    print(torch.allclose(img_latent, img_latent_pad))

# Mix retccl and retccl tuned

In [33]:
# Open retccl bag latents
root_dir = BOLERO_DIR
bag_latents_file_retccl = os.path.join(root_dir, "bag_latents_gs256_retccl.pt")
bag_latents_retccl = torch.load(bag_latents_file_retccl)
bag_latents_file_retccl_tuned = os.path.join(root_dir, "bag_latents_gs256_retccl_tuned.pt")
bag_latents_retccl_tuned = torch.load(bag_latents_file_retccl_tuned)

# Avg the two into a new dict: bag_latents_retccl_hybrid
bag_latents_retccl_hybrid = {}
for idx in bag_latents_retccl_tuned:
    slide_idx = int(idx.split("_")[0])
    biopsy_idx = int(idx.split("_")[1])
    non_tuned = bag_latents_retccl[slide_idx][biopsy_idx+1].cpu().unsqueeze(1)
    tuned = bag_latents_retccl_tuned[idx]
    bag_latents_retccl_hybrid[idx] = (non_tuned + tuned) / 2

# Save the hybrid latents
bag_latents_file_retccl_hybrid = os.path.join(root_dir, "bag_latents_gs256_retccl_tuned_hybrid.pt")
torch.save(bag_latents_retccl_hybrid, bag_latents_file_retccl_hybrid)